### Redis Python Operations
Python Redis operations
* Set
* Get

In [ ]:
import pandas as pd
import redis


In [2]:
df = pd.read_parquet("../data/df.parquet")


In [3]:
df.head(10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047,United Kingdom


In [4]:
df.shape


(397924, 8)

In [5]:
con = redis.StrictRedis(host='localhost', port=6379, db=1, password=None)
con.ping()


True

In [6]:
con.info()


{'redis_version': '7.0.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': 'eaa8c98212b8004a',
 'redis_mode': 'standalone',
 'os': 'Linux 5.10.102.1-microsoft-standard-WSL2 x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '10.2.1',
 'process_id': 1,
 'process_supervised': 'no',
 'run_id': '4e914fa8b38b90f7c64f20305fcfca80516cd29b',
 'tcp_port': 6379,
 'server_time_usec': 1654969309082855,
 'uptime_in_seconds': 7020,
 'uptime_in_days': 0,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 10802141,
 'executable': '/data/redis-server',
 'config_file': '',
 'io_threads_active': 0,
 'connected_clients': 1,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 20480,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'clients_in_timeout_table': 0,
 'used_memory': 1571384,
 'used_memory_human': '1.50M',
 'used_memory

In [7]:
tmp = df.groupby("CustomerID")["InvoiceNo"].unique().to_frame().reset_index()
tmp.head()


,CustomerID,InvoiceNo
0,12346,[541431]
1,12347,"[537626, 542237, 549222, 556201, 562032, 57351..."
2,12348,"[539318, 541998, 548955, 568172]"
3,12349,[577609]
4,12350,[543037]


In [8]:
tmp.tail()

,CustomerID,InvoiceNo
4334,18280,[545712]
4335,18281,[556464]
4336,18282,"[562525, 580173]"
4337,18283,"[540350, 541854, 545079, 550957, 554157, 55673..."
4338,18287,"[554065, 570715, 573167]"


### Inserting data into Redis

Redis is a key-value store that is designed to be very fast and flexible. Hence I just want to insert aggregated data into Redis. \
I used *CustomerID* as key and the *InvoiceNo* as value.

In [10]:
%%time
for user in tmp.CustomerID.unique():
    con.set(str(user), tmp[tmp['CustomerID'] == user]["InvoiceNo"].to_json())


CPU times: total: 4.06 s
Wall time: 6.83 s


### Getting data from Redis


In [11]:
def get_user_results(user_id):
    con = redis.StrictRedis(host='localhost', port=6379, db=1, password=None)
    res = con.get(user_id)
    return res.decode()


In [12]:
%%time
get_user_results(18283)


CPU times: total: 0 ns
Wall time: 5 ms


'{"4337":["540350","541854","545079","550957","554157","556731","557956","560025","560032","565579","573093","575668","575675","578262","579673","580872"]}'

In [13]:
%%time
tmp[tmp.CustomerID == "18283"]

CPU times: total: 0 ns
Wall time: 1 ms


,CustomerID,InvoiceNo
4337,18283,"[540350, 541854, 545079, 550957, 554157, 55673..."


In [14]:
%%timeit -n 10 -r 10
get_user_results(18283)

3.93 ms ± 252 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [15]:
%%timeit -n 10 -r 10
tmp[tmp.CustomerID == "18283"]

561 µs ± 49.8 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
